# Introdução a Pandas para Ciência de Dados - Conceitos intermediários

Neste notebook apresentamos outros conceitos importantes do Pandas. 


## Inicializando DataFrames a partir de dicionários

Uma das formas mais fácies de se inicializar um DataFrame é usando dicionários. Uma lista de dicionários pode ser usada para compor as linhas do DataFrame como no exemplo abaixo:

In [1]:
import pandas as pd

ap1 = {'endereco': 'Av V. Guarapuava, 1000', 'area': 45, 'aluguel': 800}
ap2 = {'endereco': 'Av Sete de Setembro, 170', 'area': 53, 'aluguel': 950}

apartamentos = [ap1, ap2]

df = pd.DataFrame(apartamentos)
df

,endereco,area,aluguel
0,"Av V. Guarapuava, 1000",45,800
1,"Av Sete de Setembro, 170",53,950


Podemos usar um dicionário para adicionar uma nova linha a um DataFrame:

In [2]:
ap3 = {'endereco': 'Av Sete de Setembro, 830', 'area': 35, 'aluguel': 850}

df = df.append([ap3])
df

,endereco,area,aluguel
0,"Av V. Guarapuava, 1000",45,800
1,"Av Sete de Setembro, 170",53,950
0,"Av Sete de Setembro, 830",35,850


In [3]:
# usando o endereço como índice para os próximos exemplos
df = df.set_index('endereco')
df

,area,aluguel
endereco,,
"Av V. Guarapuava, 1000",45,800
"Av Sete de Setembro, 170",53,950
"Av Sete de Setembro, 830",35,850


## Iterando sobre um DataFrame

Podemos usar o comando `for` para analisar um DataFrame linha por linha. Para isto usamos o método `iterrows`. O exemplo abaixo calcula a média dos valores dos aluguéis:

In [4]:
total = 0
for indice, apartamento in df.iterrows():
    print("Aluguel {}: {}".format(indice, apartamento['aluguel']))
    total = total + apartamento['aluguel']
    
media = total/len(df)
print("Média:", media)

Aluguel Av V. Guarapuava, 1000: 800
Aluguel Av Sete de Setembro, 170: 950
Aluguel Av Sete de Setembro, 830: 850
Média: 866.6666666666666


## Filtrando linhas com o método query()

Uma forma conveniente de filtrar linhas é usando o método query. No exemplo abaixo selecionamos apenas as linhas com área maior que 40 e aluguel menor que 900.

In [5]:
df.query("area > 40 and aluguel < 900")

,area,aluguel
endereco,,
"Av V. Guarapuava, 1000",45,800


O comando acima é equivalente a:

In [6]:
df[(df['area'] > 40) & (df['aluguel'] < 900)]

,area,aluguel
endereco,,
"Av V. Guarapuava, 1000",45,800


## Filtrando linhas com expressões regulares

Expressões regulares permitem casar e manipular strings de uma forma flexível. Por exemplo, abaixo filtramos apenas as linhas com endereço começando por 'Av S' e terminando com '70'. Mais detalhes e outros exemplos de uso de expressões regulares podem ser encontrados neste [tutorial](https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/).

In [7]:
df_full = df.reset_index()

df_full

,endereco,area,aluguel
0,"Av V. Guarapuava, 1000",45,800
1,"Av Sete de Setembro, 170",53,950
2,"Av Sete de Setembro, 830",35,850


In [8]:
df_full[df_full['endereco'].str.match('^Av S.*70$')]

,endereco,area,aluguel
1,"Av Sete de Setembro, 170",53,950


## Blocos básicos de um DataFrame

Um bloco básico na construção de um DataFrame é a Série (Series). Cada coluna de um DataFrame é uma Series. Uma Series contém um nome, um índice e uma lista (array) de valores. Veja os exemplos abaixo baseados na coluna *aluguel*:

In [9]:
coluna_aluguel = df['aluguel']

# Verificando o tipo da coluna
type(coluna_aluguel)

pandas.core.series.Series

Índice:

In [10]:
coluna_aluguel_index = coluna_aluguel.index

# Mostrando o tipo e valores do índice
print(type(coluna_aluguel_index))
print(coluna_aluguel_index)

<class 'pandas.core.indexes.base.Index'>
Index(['Av V. Guarapuava, 1000', 'Av Sete de Setembro, 170',
       'Av Sete de Setembro, 830'],
      dtype='object', name='endereco')


Valores:

In [11]:
coluna_aluguel_valores = coluna_aluguel.values

# Mostrando o tipo e valores do índice
print(type(coluna_aluguel_valores))
print(coluna_aluguel_valores)

<class 'numpy.ndarray'>
[800 950 850]


Como pôde ser visto no exemplo acima, o tipo da coluna aluguéis é `numpy.ndarray`. O Numpy é uma bilioteca de estruturas de dados e operações matemáticas. Abaixo importamos o pacote numpy e usamos uma de suas funções nos valores da coluna:

In [12]:
import numpy as np

np.mean(coluna_aluguel_valores)

866.6666666666666

In [13]:
# Reiniciando índice e adicionando valores para os próximos exemplos
df = df.reset_index()
df = df.append([{'endereco': 'Av Sete de Setembro, 730', 'area': np.NaN, 'aluguel': 775}]).reset_index(drop=True)
df

,endereco,area,aluguel
0,"Av V. Guarapuava, 1000",45.0,800
1,"Av Sete de Setembro, 170",53.0,950
2,"Av Sete de Setembro, 830",35.0,850
3,"Av Sete de Setembro, 730",NaN,775


## Método apply(), eixos de um DataFrame, função *lambda*

O método `apply()` aplica uma função a colunas ou linhas de um DataFrame. No exemplo abaixo definimos uma função que conta o número de valores nulos em uma coluna. Esta função é então aplicada no DataFrame exibido na célula anterior.

In [14]:
def conta_nan(coluna):
    return coluna.isna().sum()

df.apply(conta_nan)

endereco    0
area        1
aluguel     0
dtype: int64

Uma função também pode ser aplicada sobre valores das linhas de um DataFrame. No caso abaixo, definimos a função `aumenta_aluguel()` e a aplicamos às linhas do DataFrame para criar uma nova coluna com um valor maior para o aluguel. Para especificar que precisamos aplicar a função às linhas, definimos o parâmetro `axis=1`.

In [15]:
def aumenta_aluguel(linha):
    return linha['aluguel'] + linha['aluguel']*0.1

df['novo aluguel'] = df.apply(aumenta_aluguel, axis=1)

df

,endereco,area,aluguel,novo aluguel
0,"Av V. Guarapuava, 1000",45.0,800,880.0
1,"Av Sete de Setembro, 170",53.0,950,1045.0
2,"Av Sete de Setembro, 830",35.0,850,935.0
3,"Av Sete de Setembro, 730",NaN,775,852.5


O Python possui o conceito de "função anônima", ou "função lambda". Este recurso é útil para especificar um função simples sem precisar defini-la. Abaixo construímos uma função lambda que retorna apenas a parte do número do endereço de um apartamento.

In [16]:
df['numero ap.'] = df.apply(lambda x: x['endereco'].split(',')[1], axis=1)
df

,endereco,area,aluguel,novo aluguel,numero ap.
0,"Av V. Guarapuava, 1000",45.0,800,880.0,1000
1,"Av Sete de Setembro, 170",53.0,950,1045.0,170
2,"Av Sete de Setembro, 830",35.0,850,935.0,830
3,"Av Sete de Setembro, 730",NaN,775,852.5,730


O código acima é equivalente à sequência abaixo:

In [17]:
def obtem_numero(linha):
    numero = linha['endereco'].split(',')[1]
    return numero

df['numero ap.'] = df.apply(obtem_numero, axis=1)
df

,endereco,area,aluguel,novo aluguel,numero ap.
0,"Av V. Guarapuava, 1000",45.0,800,880.0,1000
1,"Av Sete de Setembro, 170",53.0,950,1045.0,170
2,"Av Sete de Setembro, 830",35.0,850,935.0,830
3,"Av Sete de Setembro, 730",NaN,775,852.5,730


## Tratando DataFrames grandes

Muitas vezes precisamos processar dados que não cabem na memória do computador ou que demandam procedimentos complexos que deixam o processamento lento. Descrevemos aqui algumas técnicas para amenisar este tipo de problema. Vamos utilizar o dataset de reclamações, inicialmente contendo cerca de 7000 linhas (que é uma quantidade pequena, mas suficiente para os exemplos).

In [18]:
# lê o arquivo CSV
datafile = '../data/2017-02-01_156_-_Base_de_Dados_sample.csv'
df = pd.read_csv(datafile, sep=';', encoding='latin-1')

print("Total de linhas: ", len(df))

df.head()

Total de linhas:  7013


,SOLICITACAO,TIPO,ORGAO,DATA,HORARIO,ASSUNTO,SUBDIVISAO,DESCRICAO,LOGRADOURO_ASS,BAIRRO_ASS,REGIONAL_ASS,MEIO_RESPOSTA,OBSERVACAO,SEXO,BAIRRO_CIDADAO,REGIONAL_CIDADAO,DATA_NASC,TIPO_CIDADAO,ORGAO_RESP,RESPOSTA_FINAL
0,6669771,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,09/11/2016,21:02:44,ABORDAGEM SOCIAL DE RUA - ADULTO,PESSOAS/FAMÍLIAS EM DESABRIGO NA RUA,ABORDAGEM SOCIAL DE RUA - ADULTO - PESSOAS/FAM...,"ANDRE DE BARROS, 0",CENTRO,Unidade Regional Matriz,NENHUM,NÃO SOUBE INFORMAR NUMERAÇÃO PREDIAL,M,NaN,NaN,NaN,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,Abordagem realizada. Pessoa foi orientada quan...
1,6718351,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,27/12/2016,00:40:04,ABORDAGEM SOCIAL DE RUA - ADULTO,PERDIDA/DESORIENTADA,ABORDAGEM SOCIAL DE RUA - ADULTO - PERDIDA/DES...,"NILO CAIRO, 0",CENTRO,Unidade Regional Matriz,NENHUM,"NÃO SOUBE INFORMAR NUMERAÇÃO PREDIAL, RELATA E...",M,NaN,NaN,NaN,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,"EM ABORDAGEM REALIZADA, NÃO FOI ENCONTRADA A P..."
2,6702371,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,07/12/2016,14:09:42,ABORDAGEM SOCIAL DE RUA - CRIANÇA,ALCOOLIZADAS/DROGADAS,ABORDAGEM SOCIAL DE RUA - CRIANÇA - ALCOOLIZAD...,"VISCONDE DE NACAR, 1210",CENTRO,Unidade Regional Matriz,NENHUM,NaN,F,NaN,NaN,NaN,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,"- EM ABORDAGEM REALIZADA, NÃO FOI ENCONTRADA A..."
3,6718692,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,27/12/2016,13:28:27,ABORDAGEM SOCIAL DE RUA - ADULTO,PESSOAS/FAMÍLIAS EM DESABRIGO NA RUA,ABORDAGEM SOCIAL DE RUA - ADULTO - PESSOAS/FAM...,"MARECHAL DEODORO, 0",CENTRO,Unidade Regional Matriz,NENHUM,NaN,M,NaN,NaN,NaN,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,A pessoa da solicitação já foi abordada nesta ...
4,6704503,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,09/12/2016,01:28:16,ABORDAGEM SOCIAL DE RUA - ADULTO,DORMINDO/CAÍDAS NA RUA,ABORDAGEM SOCIAL DE RUA - ADULTO - DORMINDO/CA...,"MARECHAL DEODORO, 0",CENTRO,Unidade Regional Matriz,NENHUM,NÃO SOUBE INFORMAR NUMERAÇÃO PREDIAL.,M,NaN,NaN,04/06/1997,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,A pessoa da solicitação já faz parte da rede d...


Uma forma de reduzir o tamanho de um DataFrame é fazer uma amostragem das linhas. Abaixo fazemos uma amostragem aleatória de 1000 linhas:

In [19]:
df_sample = df.sample(1000)

print(len(df_sample))

1000


O problema da estratégia de amostragem acima é que os dados já estavam na memória. Portanto, esta estratégia não funcionaria se os dados fossem maior que a memória disponível. Nestes casos precisamos fazer a amostragem no momento da leitura do arquivo de dados.

O código abaixo contrói uma lista aleatória de índices de linhas para serem ignoradas. Quando o comando `pd.read_csv()` é chamado, ele carrega na memória somente as linhas que não aparecem na lista gerada (que foi passada no parâmetro *skiprows*).

In [20]:
import random

# Definindo uma semente de geração de números aleatórios para que a seleção seja a mesma em múltiplas execuções do código
random.seed(42)

# Conta linhas do arquivo de entrada
num_linhas = sum(1 for l in open(datafile, encoding='latin-1'))

# Define a proporção dos dados a se manter
proporcao = 0.1

# calcula o tamanho desejado da amostragem
novo_tamanho = int(num_linhas * proporcao)

# define os valores de índice aleatórios que serão ignorados
skip_idx = random.sample(range(1, num_linhas), num_linhas - novo_tamanho)

# Lê os dados pulando as linhas definidas
df = pd.read_csv(datafile, sep=';', skiprows=skip_idx, encoding='latin-1')

print("Total de linhas: ", len(df))

Total de linhas:  700


Outra possibilidade é ler os dados em "lotes", processando um número determinado de linhas de cada vez. O código abaixo define o tamanho dos pedaços a serem lidos (no caso 30 linhas). Estes pedaços são lidos um de cada vez e cada um recebe um tratamento de limpeza de dados (linhas com valores nulos são eliminadas). Ao fim, apenas cerca de 3600 linhas restaram no DataFrame contruído pelo procedimento.

In [21]:
chunksize = 30

df_limpa = pd.DataFrame()

for df_chunk in pd.read_csv(datafile, chunksize=chunksize, sep=';', encoding='latin-1'):
    # limpeza de dados no chunk atual:
    df_chunk = df_chunk.dropna(how='any', axis=0)
    df_limpa = pd.concat([df_limpa, df_chunk])

    
print(len(df_limpa))
df_limpa.head()

3640


,SOLICITACAO,TIPO,ORGAO,DATA,HORARIO,ASSUNTO,SUBDIVISAO,DESCRICAO,LOGRADOURO_ASS,BAIRRO_ASS,REGIONAL_ASS,MEIO_RESPOSTA,OBSERVACAO,SEXO,BAIRRO_CIDADAO,REGIONAL_CIDADAO,DATA_NASC,TIPO_CIDADAO,ORGAO_RESP,RESPOSTA_FINAL
115,6703815,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,08/12/2016,13:43:09,ABORDAGEM SOCIAL DE RUA - ADULTO,DORMINDO/CAÍDAS NA RUA,ABORDAGEM SOCIAL DE RUA - ADULTO - DORMINDO/CA...,"DESEMBARGADOR MOTTA, 0",BATEL,Unidade Regional Matriz,TELEFONE,NÃO SOUBE INFORMAR SE O RAPAZ ESTÁ BEM NEM A N...,F,BATEL,Unidade Regional Matriz,11/01/1982,CIDADÃO,FUNDAÇÃO DE AÇÃO SOCIAL,"EM ABORDAGEM REALIZADA, NÃO FOI ENCONTRADA A P..."
118,6699104,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,05/12/2016,14:53:29,ILUMINAÇÃO PÚBLICA,MANUTENÇÃO DE LUMINÁRIAS,ILUMINAÇÃO PÚBLICA - MANUTENÇÃO DE LUMINÁRIAS,"PADRE AGOSTINHO, 781",MERCES,Unidade Regional Matriz,EMAIL,NÃO SOUBE INFORMAR A ID DO POSTE. EM FRENTE A ...,M,MERCES,Unidade Regional Matriz,11/11/1948,CIDADÃO,SECRETARIA MUNICIPAL DE OBRAS PÚBLICAS,SOLICITACAO ATENDIDA
119,6718601,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,27/12/2016,11:48:48,SEMÁFORO,MANUTENÇÃO DE GRUPOS FOCAIS OU COLUNAS (EXCETO...,SEMÁFORO - MANUTENÇÃO DE GRUPOS FOCAIS OU COLU...,"MARECHAL FLORIANO PEIXOTO, 0",CENTRO,Unidade Regional Matriz,NENHUM,NÃO SOUBE INFORMAR NÚMERO PREDIAL.,F,BATEL,Unidade Regional Matriz,05/01/1958,CIDADÃO,SECRETARIA MUNICIPAL DE TRÂNSITO,Informamos que uma ordem de serviço foi emitid...
122,6759593,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,29/01/2017,20:44:14,TRÂNSITO,VEÍCULO ESTACIONADO - CALÇADA,TRÂNSITO - VEÍCULO ESTACIONADO - CALÇADA,"VISCONDE DE GUARAPUAVA, 1447",ALTO DA RUA XV,Unidade Regional Matriz,EMAIL,NÃO SOUBE INFORMAR DADOS DOS VEÍCULOS.,M,ALTO DA RUA XV,Unidade Regional Matriz,23/10/1991,CIDADÃO,SECRETARIA MUNICIPAL DE TRÂNSITO,"encaminhado viatura 222, no local às 21:18h, c..."
123,6677442,SOLICITAÇÃO,INSTITUTO DAS CIDADES INTELIGENTES,17/11/2016,11:21:12,COLETA,RESÍDUOS VEGETAIS DE JARDIM,COLETA - RESÍDUOS VEGETAIS DE JARDIM,"VISCONDE DE GUARAPUAVA, 646",ALTO DA RUA XV,Unidade Regional Matriz,EMAIL,TRANSVERSAL 2: RUA GERMANO MEYER. NÃO INFORMOU...,F,ALTO DA RUA XV,Unidade Regional Matriz,07/07/1979,CIDADÃO,SECRETARIA MUNICIPAL DO MEIO AMBIENTE,"O contrato de Coleta de Vegetais, caliças e en..."
